In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
torch.manual_seed(42)

In [ ]:
queen_train = np.load('queen_train.npy')
nonqueen_train = np.load('nonqueen_train.npy')

queen_val = np.load('queen_val.npy')
nonqueen_val = np.load('nonqueen_val.npy')

queen_test = np.load('queen_test.npy')
nonqueen_test = np.load('nonqueen_test.npy')

In [ ]:
class BeeDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
x_train = np.vstack((queen_train, nonqueen_train))
ones = np.ones(len(queen_train))
zeros = np.zeros(len(nonqueen_train))
y_train = np.concatenate((ones, zeros))

x_val = np.vstack((queen_val, nonqueen_val))
ones = np.ones(len(queen_val))
zeros = np.zeros(len(nonqueen_val))
y_val = np.concatenate((ones, zeros))

x_test = np.vstack((queen_test, nonqueen_test))
ones = np.ones(len(queen_test))
zeros = np.zeros(len(nonqueen_test))
y_test = np.concatenate((ones, zeros))

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
train_dataset = BeeDataset(x_train, y_train, transform=transform)
val_dataset = BeeDataset(x_val, y_val, transform=transform)
test_dataset = BeeDataset(x_test, y_test, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
version = 'b0'  
if version == 'b0':
    model = models.efficientnet_b0(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
elif version == 'b1':
    model = models.efficientnet_b1(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
elif version == 'b2':
    model = models.efficientnet_b2(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
elif version == 'b3':
    model = models.efficientnet_b3(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
elif version == 'b4':
    model = models.efficientnet_b4(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
elif version == 'b5':
    model = models.efficientnet_b5(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
elif version == 'b6':
    model = models.efficientnet_b6(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 56, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
elif version == 'b7':
    model = models.efficientnet_b7(weights='IMAGENET1K_V1')
    model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [ ]:
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

In [ ]:
def initialize_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

In [ ]:
model.apply(initialize_weights)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs = 100
patience = 20
best_val_loss = float('inf')
early_stop_counter = 0

val_accs = []
train_accs = []
train_losses = []
val_losses = []

best_model_wts = model.state_dict()

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_corrects = 0

    for train_images, train_labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        train_images = train_images.to(device)
        train_labels = train_labels.long()
        train_labels = train_labels.to(device)

        optimizer.zero_grad()

        outputs = model(train_images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, train_labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * train_images.size(0)
        train_corrects += torch.sum(preds == train_labels.data)

    train_loss = train_loss / len(train_dataset)
    train_losses.append(train_loss)
    train_acc = train_corrects.double() / len(train_dataset)
    train_accs.append(train_acc)

    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation"):
            val_images = val_images.to(device)
            val_labels = val_labels.long()
            val_labels = val_labels.to(device)

            val_outputs = model(val_images)
            _, preds = torch.max(val_outputs, 1)
            loss = criterion(val_outputs, val_labels)

            val_loss += loss.item() * val_images.size(0)
            val_corrects += torch.sum(preds == val_labels.data)

    val_loss = val_loss / len(val_dataset)
    val_losses.append(val_loss)
    val_acc = val_corrects.double() / len(val_dataset)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss}, Train Acc: {train_acc}, Val Loss: {val_loss}, Val Acc: {val_acc}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = model.state_dict() 
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered.")
            model.load_state_dict(best_model_wts)  
            break

In [ ]:
train_losses = [loss.cpu().item() if isinstance(loss, torch.Tensor) else loss for loss in train_losses]
val_losses = [loss.cpu().item() if isinstance(loss, torch.Tensor) else loss for loss in val_losses]
train_accs = [acc.cpu().item() if isinstance(acc, torch.Tensor) else acc for acc in train_accs]
val_accs = [acc.cpu().item() if isinstance(acc, torch.Tensor) else acc for acc in val_accs]

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Train vs Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Train vs Validation Accuracy')

plt.show()


In [ ]:
model.eval()
test_corrects = 0
with torch.no_grad():
    for test_images, test_labels in tqdm(test_dataloader, desc="Testing"):
        test_images = test_images.to(device)
        test_labels = test_labels.long()
        test_labels = test_labels.to(device)

        test_outputs = model(test_images)
        _, preds = torch.max(test_outputs, 1)

        test_corrects += torch.sum(preds == test_labels.data)

test_acc = test_corrects.double() / len(test_dataset)
print(f'Test Accuracy: {test_acc:.4f}')